In [1]:
import os
from torch import nn
# from torchvision.models import VGG16_Weights
import torchvision.models as models
import torch
import ssl
import numpy as np
ssl._create_default_https_context = ssl._create_unverified_context # 这行代码为了  下载预训练权重时不出网络问题

cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")
print('Using {} device'.format(device))

Using cuda:0 device


In [2]:
seed = 1008
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [6]:
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

class myBigBatchDataset_highMemory(Dataset):
  def __init__(self,data_set_path,train):
    self.labelList=[]
    self.imgList=[]
    trainMap={}

    if train:
      train=1
    else:
      train=0

    with open(data_set_path+r"train_test_split.txt",'r') as trainList:
      trainList=trainList.readlines()
    for line in trainList:
      listt=line.split()
      trainMap[int(listt[0])]=int(listt[1])

    with open(data_set_path+r"images.txt",'r') as imgNames:
      imgNames=imgNames.readlines()

    with open(data_set_path+r"image_class_labels.txt",'r') as image_class_labels:
      image_class_labels=image_class_labels.readlines()


    for i in range(len(trainList)):
      if trainMap[i+1]!=train:
        continue
      imgName=imgNames[i].split()[1]
      image_class_label=int(image_class_labels[i].split()[1])-1
      imgPath=data_set_path+os.sep+r"images/"+os.sep+imgName
      trans=transforms.ToTensor()

      with Image.open(imgPath).convert('RGB')  as op:# 貌似这是jpg的图所以没有透明通道，所以.convert('RGB')与否不重要
          op=op.resize((224,224))
          op=trans(op)
      # print("image_class_label",image_class_label)

      self.labelList.append(image_class_label)
      self.imgList.append(op)


  def __getitem__(self, index):
    return self.labelList[index],self.imgList[index]

  def __len__(self):
    return len(self.labelList)


In [7]:
trainDataSet=myBigBatchDataset_highMemory(r"./CUB_200_2011/CUB_200_2011/",True)
testDataSet=myBigBatchDataset_highMemory(r"./CUB_200_2011/CUB_200_2011/",False)

In [8]:
trainDataLoader=DataLoader(trainDataSet,batch_size=50,shuffle=True)
testDataLoader=DataLoader(testDataSet,batch_size=100,shuffle=False)

In [9]:
import torch.nn.functional as F
class vgg16FineTuneModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model=models.vgg16(pretrained=True)
        self.model.classifier[-1]=nn.Linear(in_features=4096, out_features=200)

    def forward(self, x):
        x=self.model(x)
        return x

    def frz(self,fz):
        if fz:
            for param in self.model.parameters():
                param.requires_grad=False
            for param in self.model.classifier[-1].parameters():
                param.requires_grad=True
        else:
            for param in self.model.parameters():
                param.requires_grad=True

In [10]:
md=vgg16FineTuneModel()

D:\anacccnew\envs\mul2\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
D:\anacccnew\envs\mul2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
from torch import optim

def train(model,train_loader,epoch,learning_rate=0.1,fz=True):
    optimizer = optim.SGD(model.parameters(), lr=learning_rate,momentum=0.7)
    lossFunction=nn.CrossEntropyLoss()
    model.to(device)
    model.train()
    model.frz(fz)
    for i in range(epoch):
        wrongNum=0
        rightNum=0
        print("epoch",i+1,"/",epoch)
        for batch_idx,(label,data) in enumerate(train_loader):
            data=data.to(device)
            label=label.to(device)
            optimizer.zero_grad()

            output=model.forward(data)

            loss=lossFunction(output,label)
            loss.backward()
            optimizer.step()

            pre=output.argmax(dim=1)
            summ=torch.eq(pre,label).sum()
            summ=int(summ)
            rightNum+=summ
            wrongNum+=(len(label)-summ)

            if batch_idx%16==0:
                print("loss","%.6f"%float(loss),"rightNum",rightNum,"wrongNum",wrongNum,"Acc:%.6f%%"%(100*rightNum/(rightNum+wrongNum)))

        torch.save(model, 'vgg16finetuneBig.pth')
        print("TOTALrightNum",rightNum,"TOTALwrongNum",wrongNum,"Acc:%.6f%%"%(100*rightNum/(rightNum+wrongNum)))
        print("save")

In [20]:
def test(model,val_loader): # 定义测试函数
    model.eval()
    correct = []
    with torch.no_grad(): # 使模型在运行时不进行梯度跟踪，可以减少模型运行时对内存的占用。
        for i,data in enumerate(val_loader):
            y,x = data
            x = x.to(device)
            y = y.to(device)
            y_hat = model(x)

            pred = y_hat.max(1,keepdim=True)[1] # 获取预测结果
            correct.append(pred.eq(y.view_as(pred)).sum().item()/pred.shape[0]) # 收集精确度
    print('\nTest:Average Accuracy:({:,.6f}%)\n'.format(np.mean(correct)*100)) # 输出测试结果

In [15]:
# torch.cuda.empty_cache()
train(md,trainDataLoader,epoch=3,learning_rate=0.1,fz=True)

epoch 1 / 2
loss 5.354841 rightNum 17 wrongNum 33 Acc:34.000000%
loss 4.381519 rightNum 306 wrongNum 544 Acc:36.000000%
loss 5.773132 rightNum 566 wrongNum 1084 Acc:34.303030%
loss 4.770171 rightNum 828 wrongNum 1622 Acc:33.795918%
loss 5.717267 rightNum 1055 wrongNum 2195 Acc:32.461538%
loss 5.312715 rightNum 1298 wrongNum 2752 Acc:32.049383%
loss 6.733914 rightNum 1525 wrongNum 3325 Acc:31.443299%
loss 6.831774 rightNum 1766 wrongNum 3884 Acc:31.256637%
TOTALrightNum 1869 TOTALwrongNum 4125 Acc:31.181181%
save
epoch 2 / 2
loss 4.405069 rightNum 16 wrongNum 34 Acc:32.000000%
loss 4.107169 rightNum 345 wrongNum 505 Acc:40.588235%
loss 5.709774 rightNum 626 wrongNum 1024 Acc:37.939394%
loss 4.222543 rightNum 924 wrongNum 1526 Acc:37.714286%
loss 4.755603 rightNum 1197 wrongNum 2053 Acc:36.830769%
loss 6.934615 rightNum 1484 wrongNum 2566 Acc:36.641975%
loss 7.122550 rightNum 1767 wrongNum 3083 Acc:36.432990%
loss 5.140089 rightNum 2037 wrongNum 3613 Acc:36.053097%
TOTALrightNum 2156 TOT

In [16]:
test(md,testDataLoader)


Test:Average Accuracy:(31%)



In [17]:
train(md,trainDataLoader,epoch=10,learning_rate=0.0002,fz=False)

epoch 1 / 10
loss 4.299787 rightNum 23 wrongNum 27 Acc:46.000000%
loss 2.861226 rightNum 402 wrongNum 448 Acc:47.294118%
loss 2.128547 rightNum 807 wrongNum 843 Acc:48.909091%
loss 2.041297 rightNum 1212 wrongNum 1238 Acc:49.469388%
loss 1.690367 rightNum 1642 wrongNum 1608 Acc:50.523077%
loss 1.843529 rightNum 2082 wrongNum 1968 Acc:51.407407%
loss 1.759590 rightNum 2511 wrongNum 2339 Acc:51.773196%
loss 2.053535 rightNum 2937 wrongNum 2713 Acc:51.982301%
TOTALrightNum 3133 TOTALwrongNum 2861 Acc:52.268936%
save
epoch 2 / 10
loss 1.437905 rightNum 27 wrongNum 23 Acc:54.000000%
loss 1.550005 rightNum 505 wrongNum 345 Acc:59.411765%
loss 1.574197 rightNum 983 wrongNum 667 Acc:59.575758%
loss 1.400715 rightNum 1450 wrongNum 1000 Acc:59.183673%
loss 1.626974 rightNum 1934 wrongNum 1316 Acc:59.507692%
loss 1.453137 rightNum 2412 wrongNum 1638 Acc:59.555556%
loss 1.571787 rightNum 2880 wrongNum 1970 Acc:59.381443%
loss 1.847677 rightNum 3349 wrongNum 2301 Acc:59.274336%
TOTALrightNum 3549 T

In [18]:
# torch.cuda.empty_cache()
test(md, testDataLoader)


Test:Average Accuracy:(46%)



In [19]:
train(md,trainDataLoader,epoch=5,learning_rate=0.00005,fz=False)
test(md, testDataLoader)

epoch 1 / 5
loss 0.603360 rightNum 42 wrongNum 8 Acc:84.000000%
loss 0.975215 rightNum 668 wrongNum 182 Acc:78.588235%
loss 0.743293 rightNum 1292 wrongNum 358 Acc:78.303030%
loss 0.491850 rightNum 1911 wrongNum 539 Acc:78.000000%
loss 0.853033 rightNum 2540 wrongNum 710 Acc:78.153846%
loss 0.528115 rightNum 3166 wrongNum 884 Acc:78.172840%
loss 0.619024 rightNum 3797 wrongNum 1053 Acc:78.288660%
loss 0.798643 rightNum 4423 wrongNum 1227 Acc:78.283186%
TOTALrightNum 4683 TOTALwrongNum 1311 Acc:78.128128%
save
epoch 2 / 5
loss 0.793550 rightNum 39 wrongNum 11 Acc:78.000000%
loss 0.600752 rightNum 645 wrongNum 205 Acc:75.882353%
loss 0.443632 rightNum 1288 wrongNum 362 Acc:78.060606%
loss 0.651513 rightNum 1917 wrongNum 533 Acc:78.244898%
loss 0.616753 rightNum 2555 wrongNum 695 Acc:78.615385%
loss 0.740887 rightNum 3197 wrongNum 853 Acc:78.938272%
loss 0.922331 rightNum 3812 wrongNum 1038 Acc:78.597938%
loss 1.024472 rightNum 4414 wrongNum 1236 Acc:78.123894%
TOTALrightNum 4681 TOTALwro

In [21]:
test(md, testDataLoader)


Test:Average Accuracy:(47.045488%)

